In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('Johns work/csv_file/Final_Data_Exploration_df.csv')

In [2]:
df = df.drop(['Unnamed: 0', 'index'], axis = 1)

In [3]:
df = df.drop(['State', 'City', 'dup', 'flights', 'cancelled', 'origin_city_name', 'dest_city_name', 'tail_num', 'Region'], axis = 1)
df.sort_values(['fl_date']).reset_index(drop = True)

,fl_date,mkt_unique_carrier,branded_code_share,op_unique_carrier,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,dep_delay,...,taxi_in,crs_arr_time,arr_time,arr_delay,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,Weather
0,2018-12-01,AA,AA_CODESHARE,YV,5948,14730,11298,1229,1337.0,68.0,...,9.0,1355,1501.0,66.0,0,146,144.0,123.0,733,Rainy
1,2018-12-01,AA,AA_CODESHARE,MQ,3534,13930,11066,729,725.0,-4.0,...,7.0,945,931.0,-14.0,0,76,66.0,46.0,296,Rainy
2,2018-12-01,AA,AA_CODESHARE,MQ,3535,13459,13930,815,810.0,-5.0,...,17.0,851,831.0,-20.0,0,96,81.0,58.0,303,Rainy
3,2018-12-01,AA,AA_CODESHARE,MQ,3537,11298,11067,1305,1337.0,32.0,...,2.0,1502,1525.0,23.0,0,117,108.0,86.0,692,Rainy
4,2018-12-01,AA,AA_CODESHARE,MQ,3999,11193,13930,1528,1528.0,0.0,...,15.0,1552,1550.0,-2.0,0,84,82.0,57.0,264,Rainy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478837,2019-04-19,AA,AA_CODESHARE,OH,5296,14100,15096,1825,1857.0,32.0,...,3.0,1942,2007.0,25.0,0,77,70.0,51.0,228,Cloudy
478838,2019-04-19,AA,AA_CODESHARE,OH,5297,15295,11057,624,620.0,-4.0,...,16.0,814,813.0,-1.0,0,110,113.0,82.0,466,Rainy
478839,2019-04-19,AA,AA_CODESHARE,OH,5298,11057,14193,940,939.0,-1.0,...,2.0,1031,1039.0,8.0,0,111,120.0,81.0,488,Rainy
478840,2019-04-19,AA,AA_CODESHARE,OH,5283,14307,11278,1937,1929.0,-8.0,...,5.0,2114,2101.0,-13.0,0,97,92.0,74.0,356,Rainy


In [4]:
#Setting up our dfs with different features
import copy
binning_df = copy.deepcopy(df)
frequency_df = copy.deepcopy(df)

# Before we can start modeling we have to do something with our categorical variables

Lets try binning mkt_unique_carrier, branded_code_share, op_unique_carrier, and airport id's and then one hot encoding

In [5]:
#Turning int columns into strings because they represent categories
to_string = ['op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id']

for value in to_string:
    df[value] = df[value].astype(str)

In [6]:
flight_feat = df.dtypes[df.dtypes == 'object'].index.tolist()
flight_feat

['fl_date',
 'mkt_unique_carrier',
 'branded_code_share',
 'op_unique_carrier',
 'op_carrier_fl_num',
 'origin_airport_id',
 'dest_airport_id',
 'Weather']

In [7]:
#Creating bins
airports_with_less_than_4000_arrivals = binning_df['dest_airport_id'].value_counts().loc[lambda x: x < 3500].keys()
airports_with_between_4000_and_9000_arrivals = binning_df['dest_airport_id'].value_counts().loc[lambda x: (x >= 3500) & (x < 8000)].keys()
airports_with_between_9000_and_12000_arrivals = binning_df['dest_airport_id'].value_counts().loc[lambda x: (x >= 8000) & (x < 12000)].keys()
airports_with_more_than_12000_arrivals = binning_df['dest_airport_id'].value_counts().loc[lambda x: x > 12000].keys()

In [8]:
#Replacing values with our created bins
binning_df.loc[binning_df['dest_airport_id'].isin(airports_with_less_than_4000_arrivals), ['dest_airport_id']] = '0 to 4000 arrivals'
binning_df.loc[binning_df['dest_airport_id'].isin(airports_with_between_4000_and_9000_arrivals), ['dest_airport_id']] = '4000 to 9000 arrivals'
binning_df.loc[binning_df['dest_airport_id'].isin(airports_with_between_9000_and_12000_arrivals), ['dest_airport_id']] = '9000 to 12000 arrivals'
binning_df.loc[binning_df['dest_airport_id'].isin(airports_with_more_than_12000_arrivals), ['dest_airport_id']] = 'More than 12000 arrivals'

In [9]:
#Checking new distribution
binning_df['dest_airport_id'].value_counts()

9000 to 12000 arrivals      140840
0 to 4000 arrivals          137709
More than 12000 arrivals    112544
4000 to 9000 arrivals        87749
Name: dest_airport_id, dtype: int64

In [10]:
#Repeating the above process with departures instead
airports_with_less_than_4000_departures = binning_df['origin_airport_id'].value_counts().loc[lambda x: x < 3500].keys()
airports_with_between_4000_and_9000_departures = binning_df['origin_airport_id'].value_counts().loc[lambda x: (x >= 3500) & (x < 8000)].keys()
airports_with_between_9000_and_12000_departures = binning_df['origin_airport_id'].value_counts().loc[lambda x: (x >= 8000) & (x < 12000)].keys()
airports_with_more_than_12000_departures = binning_df['origin_airport_id'].value_counts().loc[lambda x: x > 12000].keys()

In [11]:
binning_df.loc[binning_df['origin_airport_id'].isin(airports_with_less_than_4000_departures), ['origin_airport_id']] = '0 to 4000 departures'
binning_df.loc[binning_df['origin_airport_id'].isin(airports_with_between_4000_and_9000_departures), ['origin_airport_id']] = '4000 to 9000 departures'
binning_df.loc[binning_df['origin_airport_id'].isin(airports_with_between_9000_and_12000_departures), ['origin_airport_id']] = '9000 to 12000 departures'
binning_df.loc[binning_df['origin_airport_id'].isin(airports_with_more_than_12000_departures), ['origin_airport_id']] = 'More than 12000 departures'

In [12]:
#Checking new distribution
binning_df['origin_airport_id'].value_counts()

0 to 4000 departures          139919
9000 to 12000 departures      138237
More than 12000 departures    111095
4000 to 9000 departures        89591
Name: origin_airport_id, dtype: int64

In [13]:
#Binning mkt_unique_carrier
binning_df['mkt_unique_carrier'].value_counts()/478842
low_frequency = ['HA', 'G4', 'F9', 'NK', 'B6', 'AS']
binning_df.loc[binning_df['mkt_unique_carrier'].isin(low_frequency), ['mkt_unique_carrier']] = "'HA', 'G4', 'F9', 'NK', 'B6', 'AS'"

In [14]:
#Binning branded_code_share
binning_df['branded_code_share'].value_counts()/478842
low_frequency = ['HA', 'G4', 'F9', 'AS_CODESHARE', 'NK', 'B6', 'AS','UA', 'DL_CODESHARE']
med_frequency = ['DL', 'UA_CODESHARE', 'AA']
binning_df.loc[binning_df['branded_code_share'].isin(low_frequency), ['branded_code_share']] = "'HA', 'G4', 'F9', 'AS_CODESHARE', 'NK', 'B6', 'AS', 'UA', 'DL_CODESHARE'"
binning_df.loc[binning_df['branded_code_share'].isin(med_frequency), ['branded_code_share']] = "'DL', 'UA_CODESHARE', 'AA'"

In [15]:
#Binning op_unique_carrier
binning_df['op_unique_carrier'].value_counts()/478842
low_frequency = ['HA', 'C5', 'AX', 'G7', 'ZW', 'CP', 'G4', 'F9', 'PT', 'QX', 'EV', 'NK', 'AS', 'YV', '9E', 'OH', 'MQ', 'YX', 'B6']
binning_df.loc[binning_df['op_unique_carrier'].isin(low_frequency), ['op_unique_carrier']] = "'HA', 'C5', 'AX', 'G7', 'ZW', 'CP', 'G4', 'F9', 'PT', 'QX', 'EV', 'NK', 'AS', 'YV', '9E', 'OH', 'MQ', 'YX', 'B6'"

In [16]:
#Binning 'op_carrier_fl_num'
rare_op_flight_num = binning_df['op_carrier_fl_num'].value_counts().loc[lambda x: x < 50].keys()
uncommon_op_flight_num = binning_df['op_carrier_fl_num'].value_counts().loc[lambda x: (x >= 50) & (x <75)].keys()
common_op_flight_num = binning_df['op_carrier_fl_num'].value_counts().loc[lambda x: (x >= 75) & (x <100)].keys()
frequent_op_flight_num = binning_df['op_carrier_fl_num'].value_counts().loc[lambda x: x >= 100].keys()

In [17]:
binning_df.loc[binning_df['op_carrier_fl_num'].isin(rare_op_flight_num), ['op_carrier_fl_num']] = 'rare_op_flight_num'
binning_df.loc[binning_df['op_carrier_fl_num'].isin(uncommon_op_flight_num), ['op_carrier_fl_num']] = 'uncommon_op_flight_num'
binning_df.loc[binning_df['op_carrier_fl_num'].isin(common_op_flight_num), ['op_carrier_fl_num']] = 'common_op_flight_num'
binning_df.loc[binning_df['op_carrier_fl_num'].isin(frequent_op_flight_num), ['op_carrier_fl_num']] = 'frequent_op_flight_num'

In [18]:
for value in binning_df.columns:
    print(value)

fl_date
mkt_unique_carrier
branded_code_share
op_unique_carrier
op_carrier_fl_num
origin_airport_id
dest_airport_id
crs_dep_time
dep_time
dep_delay
taxi_out
wheels_off
wheels_on
taxi_in
crs_arr_time
arr_time
arr_delay
diverted
crs_elapsed_time
actual_elapsed_time
air_time
distance
Weather


In [19]:
binning_df['op_carrier_fl_num'].value_counts()

frequent_op_flight_num    177518
common_op_flight_num      129076
uncommon_op_flight_num    114158
rare_op_flight_num         58090
Name: op_carrier_fl_num, dtype: int64

In [20]:
#Creating dummies
cat_feats = binning_df.dtypes[binning_df.dtypes == 'object'].index.tolist()
cat_feats.pop(0)
Binning_dummies = pd.get_dummies(binning_df[cat_feats])
Binning_dummies

,"mkt_unique_carrier_'HA', 'G4', 'F9', 'NK', 'B6', 'AS'",mkt_unique_carrier_AA,mkt_unique_carrier_DL,mkt_unique_carrier_UA,mkt_unique_carrier_WN,"branded_code_share_'DL', 'UA_CODESHARE', 'AA'","branded_code_share_'HA', 'G4', 'F9', 'AS_CODESHARE', 'NK', 'B6', 'AS', 'UA', 'DL_CODESHARE'",branded_code_share_AA_CODESHARE,branded_code_share_WN,"op_unique_carrier_'HA', 'C5', 'AX', 'G7', 'ZW', 'CP', 'G4', 'F9', 'PT', 'QX', 'EV', 'NK', 'AS', 'YV', '9E', 'OH', 'MQ', 'YX', 'B6'",...,origin_airport_id_9000 to 12000 departures,origin_airport_id_More than 12000 departures,dest_airport_id_0 to 4000 arrivals,dest_airport_id_4000 to 9000 arrivals,dest_airport_id_9000 to 12000 arrivals,dest_airport_id_More than 12000 arrivals,Weather_Cloudy,Weather_Rainy,Weather_Snowy,Weather_Sunny
0,0,1,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,1,0,0
1,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0,1,0,1,0,0,0
2,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0,1,0,0,0,0,1
3,1,0,0,0,0,0,1,0,0,1,...,0,1,0,1,0,0,0,1,0,0
4,1,0,0,0,0,0,1,0,0,1,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478837,0,0,0,1,0,1,0,0,0,1,...,0,1,0,1,0,0,0,0,0,1
478838,0,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
478839,0,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
478840,0,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1


In [21]:
binning_df = pd.concat([binning_df, Binning_dummies], axis = 1)
before_bins = ['dest_airport_id', 'origin_airport_id', 'mkt_unique_carrier', 'branded_code_share', 'op_unique_carrier', 'op_carrier_fl_num']
binning_df = binning_df.drop(before_bins, axis = 1)
binning_df

,fl_date,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,...,origin_airport_id_9000 to 12000 departures,origin_airport_id_More than 12000 departures,dest_airport_id_0 to 4000 arrivals,dest_airport_id_4000 to 9000 arrivals,dest_airport_id_9000 to 12000 arrivals,dest_airport_id_More than 12000 arrivals,Weather_Cloudy,Weather_Rainy,Weather_Snowy,Weather_Sunny
0,2018-12-01,1229,1337.0,68.0,12.0,1349.0,1452.0,9.0,1355,1501.0,...,0,0,0,0,0,1,0,1,0,0
1,2018-12-01,1945,1941.0,-4.0,15.0,1956.0,2209.0,6.0,2244,2215.0,...,1,0,0,0,1,0,1,0,0,0
2,2018-12-01,1110,1107.0,-3.0,32.0,1139.0,1844.0,12.0,1855,1856.0,...,1,0,0,0,1,0,0,0,0,1
3,2018-12-01,1739,1736.0,-3.0,11.0,1747.0,1911.0,4.0,1930,1915.0,...,0,1,0,1,0,0,0,1,0,0
4,2018-12-01,930,926.0,-4.0,12.0,938.0,1607.0,7.0,1649,1614.0,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478837,2019-04-19,1335,1357.0,22.0,18.0,1415.0,1702.0,3.0,1640,1705.0,...,0,1,0,1,0,0,0,0,0,1
478838,2019-04-19,605,605.0,0.0,23.0,628.0,750.0,22.0,815,812.0,...,0,0,0,0,0,1,0,0,0,1
478839,2019-04-19,1515,1603.0,48.0,14.0,1617.0,1636.0,7.0,1619,1643.0,...,0,0,0,0,0,1,0,0,0,1
478840,2019-04-19,600,557.0,-3.0,27.0,624.0,643.0,8.0,715,651.0,...,0,0,0,0,0,1,0,0,0,1


# Lets try using frequencies instead of binning do deal with categorical variables

In [22]:
dest_airport_freq = frequency_df.groupby('dest_airport_id').size()/len(frequency_df)
frequency_df.loc[:, 'destination_id_frequency'] = frequency_df['dest_airport_id'].map(dest_airport_freq)

In [23]:
def make_frequency(Feature, New_Feature):
    '''
    This function takes in the name of the column you want to use and the name of the column you want to create
    This function returns a new column with frequencies encoded as categories
    '''
    frequency = frequency_df.groupby(Feature).size()/len(frequency_df)
    frequency_df.loc[:, New_Feature] = frequency_df[Feature].map(frequency)
     
    return

In [24]:
make_frequency('dest_airport_id', 'destination_id_frequency')
make_frequency('origin_airport_id', 'origin_id_frequency')
make_frequency('mkt_unique_carrier', 'mkt_frequency')
make_frequency('branded_code_share', 'branded_frequency')
make_frequency('op_unique_carrier', 'op_frequency')
make_frequency('op_carrier_fl_num', 'flight_num_frequency')

In [25]:
originals = ['dest_airport_id', 'origin_airport_id', 'mkt_unique_carrier', 'branded_code_share', 'op_unique_carrier', 'op_carrier_fl_num']
frequency_df = frequency_df.drop(originals, axis = 1)

In [26]:
Dummy = pd.get_dummies(frequency_df['Weather'])
frequency_df = pd.concat([frequency_df, Dummy], axis = 1)
frequency_df =  frequency_df.drop(['Weather'], axis = 1)

In [27]:
#frequency_df.to_csv('Features_with_frequencies.csv')

In [28]:
#binning_df.to_csv('Features_with_binning.csv')


# Lets find the average delays per month and add them in as features
We will first separate the fl_date column and extract the month and then use the groupby method to replace the monthly values with the mean

In [29]:
month_df = copy.deepcopy(binning_df)
month_df

,fl_date,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,...,origin_airport_id_9000 to 12000 departures,origin_airport_id_More than 12000 departures,dest_airport_id_0 to 4000 arrivals,dest_airport_id_4000 to 9000 arrivals,dest_airport_id_9000 to 12000 arrivals,dest_airport_id_More than 12000 arrivals,Weather_Cloudy,Weather_Rainy,Weather_Snowy,Weather_Sunny
0,2018-12-01,1229,1337.0,68.0,12.0,1349.0,1452.0,9.0,1355,1501.0,...,0,0,0,0,0,1,0,1,0,0
1,2018-12-01,1945,1941.0,-4.0,15.0,1956.0,2209.0,6.0,2244,2215.0,...,1,0,0,0,1,0,1,0,0,0
2,2018-12-01,1110,1107.0,-3.0,32.0,1139.0,1844.0,12.0,1855,1856.0,...,1,0,0,0,1,0,0,0,0,1
3,2018-12-01,1739,1736.0,-3.0,11.0,1747.0,1911.0,4.0,1930,1915.0,...,0,1,0,1,0,0,0,1,0,0
4,2018-12-01,930,926.0,-4.0,12.0,938.0,1607.0,7.0,1649,1614.0,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478837,2019-04-19,1335,1357.0,22.0,18.0,1415.0,1702.0,3.0,1640,1705.0,...,0,1,0,1,0,0,0,0,0,1
478838,2019-04-19,605,605.0,0.0,23.0,628.0,750.0,22.0,815,812.0,...,0,0,0,0,0,1,0,0,0,1
478839,2019-04-19,1515,1603.0,48.0,14.0,1617.0,1636.0,7.0,1619,1643.0,...,0,0,0,0,0,1,0,0,0,1
478840,2019-04-19,600,557.0,-3.0,27.0,624.0,643.0,8.0,715,651.0,...,0,0,0,0,0,1,0,0,0,1


In [30]:
#Extracting the month from the fl_date column
month = month_df['fl_date'].str.split('-', expand = True)
month = month.drop([0, 2], axis = 1)
month = month.rename({1:'month'}, axis = 1)

In [31]:
#Joining the month df to the time_df
month_df = pd.concat([month_df, month], axis = 1)
month_df = month_df.drop(['fl_date'], axis = 1)

In [32]:
#Map the mean for each month into the original df
average_monthly_delay = month_df.groupby('month')['dep_delay'].mean()
month_df.loc[:, 'average_monthly_delay'] = month_df['month'].map(average_monthly_delay)
month_df = month_df.drop(['dep_delay', 'dep_time'], axis = 1)

In [33]:
month_df = month_df.drop(['arr_time'], axis = 1)

# Lets create a feature for arrival and departure times that bins them to the time of day
4 bins will be made for each feature
* 0-6am morning
* 6-12pm late morning
* 12-6pm early evening
* 6-12am evening

In [34]:
time_df = copy.deepcopy(month_df)

In [35]:
time_df[['crs_dep_time', 'crs_arr_time']]

,crs_dep_time,crs_arr_time
0,1229,1355
1,1945,2244
2,1110,1855
3,1739,1930
4,930,1649
...,...,...
478837,1335,1640
478838,605,815
478839,1515,1619
478840,600,715


In [36]:
#Creating a function to define our time intervals
def bin_time_intervals(input_time, binned_time):
    
    ''' This function takes in the desired input column and desired outcome column as strings
        This function returns 4 bins representing time of day when the input column is in the format hhmm
    '''
        
    
    bins = [0, 600, 1200, 1800, 2400]
    names = ['12am-6am', '6am-12pm', '12pm-6pm', '6pm-12am']
    
    time_df[binned_time] = pd.cut(time_df[input_time], bins, labels = names)
    return

bin_time_intervals('crs_dep_time', 'time_of_departure')
bin_time_intervals('crs_arr_time', 'time_of_arrival')

In [37]:
# Dropping the columns we dont need anymore
time_df = time_df.drop(['crs_dep_time', 'crs_arr_time','month'], axis = 1)

In [38]:
# Creating dummies
time_dummies = pd.get_dummies(time_df[['time_of_departure', 'time_of_arrival']])

In [39]:
# Merging the dummy table with the original
time_df = pd.concat([time_df, time_dummies], axis = 1)

In [40]:
# Dropping last few columns to create the final dataframe 
time_df = time_df.drop(['time_of_arrival', 'time_of_departure', 'Weather', 'actual_elapsed_time'], axis = 1)

In [41]:
#time_df.to_csv('Final_Features.csv')

In [45]:
test_df = pd.read_csv('Johns work/csv_file/Test_file.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Johns work/csv_file/Test_file.csv'